
# Notebook representing Data Canvas Example for Trending Products

In [ ]:
# @title Inspect the schema of fashion_data.sales_forecast
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# Fetch the table
table = client.get_table('$project_id.fashion_data.sales_forecast')

# View table info
print("{} rows".format(table.num_rows))
for row in table.schema:
  print("Column {}: {}".format(row.name, row.field_type))


In [ ]:
# @title Setup bigquery client and formatting
from google.cloud import bigquery
from google.colab import data_table

project = '$project_id' # Project ID inserted based on the query results selected to explore
client = bigquery.Client(project=project)
data_table.enable_dataframe_formatter()

In [ ]:
# @title Executes the query

sql = '''SELECT
product_name,
extract(year from forecast_timestamp),
extract(month from forecast_timestamp),
sum(forecast_value)
FROM `fashion_data.sales_forecast`
where product_name in ('Denim Jacket','Denim Shacket')
and extract(month from forecast_timestamp) in (4,5,6)
group by 1,2,3
order by 4 desc'''
query = client.query(sql)
query
# @title Render the query results

job = client.get_job(query.job_id)
df = job.to_dataframe()
df


In [ ]:
# @title Executes the query

sql1 = '''    SELECT
      *
    FROM
      ML.FORECAST(MODEL `fashion_data.sales_forecast_model`,
                  STRUCT(30 AS horizon,
                         0.90 AS confidence_level)
                 )'''
query1 = client.query(sql1)
query1
# @title Render the query results

job1 = client.get_job(query1.job_id)
df1 = job1.to_dataframe()
df1


In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['forecast_timestamp']
  ys = series['forecast_value']

  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_30.sort_values('forecast_timestamp', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('product_name')):
  _plot_series(series, series_name, i)
  fig.legend(title='product_name', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('forecast_timestamp')
_ = plt.ylabel('forecast_value')

In [ ]:
# @title Inspect the schema of fashion_data.historical_sales
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# Fetch the table
table = client.get_table('$project_id.fashion_data.historical_sales')

# View table info
print("{} rows".format(table.num_rows))
for row in table.schema:
  print("Column {}: {}".format(row.name, row.field_type))


In [ ]:
# @title Inspect the schema of fashion_data.trending_product
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# Fetch the table
table = client.get_table('$project_id.fashion_data.trending_product')

# View table info
print("{} rows".format(table.num_rows))
for row in table.schema:
  print("Column {}: {}".format(row.name, row.field_type))


In [ ]:
# @title Executes the query

sql2 = '''# prompt: sales for denim jacket every week

SELECT
  EXTRACT(WEEK
  FROM
    historical_sales.order_datetime) AS week_number,
  SUM(historical_sales.total_sales_amount) AS total_sales
FROM
  `fashion_data.historical_sales` AS historical_sales
WHERE
  historical_sales.product_name = 'Denim Jacket'
GROUP BY
  1'''
query2 = client.query(sql2)
query2
# @title Render the query results

job2 = client.get_job(query2.job_id)
df2 = job2.to_dataframe()
df2


In [ ]:
from matplotlib import pyplot as plt
_df_43['week_number'].plot(kind='hist', bins=20, title='week_number')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# @title Executes the query

sql3 = '''# prompt: show me top trending products and their descriptions

SELECT
  trending_product.product_name,
  trending_product.product_description
FROM
  `fashion_data.trending_product` AS trending_product
ORDER BY
  trending_product.product_price DESC
LIMIT
  10'''
query3 = client.query(sql3)
query3
# @title Render the query results

job3 = client.get_job(query3.job_id)
df3 = job3.to_dataframe()
df3
